In [ ]:
!unzip training.zip

In [ ]:
!pip install transformers

In [ ]:
#from https://github.com/shashnkvats/Indofashionclip/blob/main/indofashion_clip.py
#with modifications


In [5]:
# the json file with filenames and captions
json_path = 'simple.json'


## The main event

The code below should create something that is compatible with huggingface transformers architecture, which means it should be easier to integrate with llm-clip

In [ ]:
import json
import os
import re
import string

from PIL import Image
import tqdm
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from transformers import CLIPModel, CLIPProcessor
from torchvision.transforms import ToTensor, Resize, Compose

# Setting cudnn benchmark
torch.backends.cudnn.benchmark = True

with open(json_path, 'r') as f:
    input_data = json.load(f)

# Load CLIP model and processor from Hugging Face

# Setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Loading model
single_gpu_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
model = torch.nn.DataParallel(single_gpu_model).to(device)   # Make model parallel

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


def clean_caption(raw_caption, max_caption_char_limit=300):
    if not raw_caption:
       return ''
    # just to be sure we're dealing with a string
    raw_caption = str(raw_caption)
    # Remove line breaks
    raw_caption = raw_caption.replace('\n', ' ')
    # Remove all other punctuation.
    raw_caption =  re.sub(r'[' + string.punctuation + r']+', ' ', raw_caption).strip()
    ok_words = []
    for act_word in raw_caption.split(' '):
        if not act_word:
           # Skip empty characters
           continue
        if len(' '.join(ok_words + [act_word])) < max_caption_char_limit:
            # Adding the act_word is ok, because it's less than our
            # character limits
            ok_words.append(act_word)
    # Combine it all into a single string.
    caption = ' '.join(ok_words)
    # truncating shouldn't be needed, but seems safer
    return caption[:max_caption_char_limit]
       

# Define a custom dataset
class image_title_dataset():
    def __init__(self, list_image_path,list_txt):
        self.image_path = list_image_path
        self.list_txt = list_txt
        self.transform = Compose([Resize((224, 224)), ToTensor()])  # Resize all images to 224x224

    def __len__(self):
        return len(self.list_txt)

    def __getitem__(self, idx):
      try:
        image = Image.open(self.image_path[idx]).convert("RGB")
        image = self.transform(image)  # Apply the transforms
      except Exception as e:
        print(f"Unable to open image at {self.image_path[idx]} due to error : {e}")
        image = torch.zeros(3, 224, 224)  # Adding a Zero tensor in case of image loading failure
        # Return an empty string as text
      return image, self.list_txt[idx] if self.list_txt[idx] else [""]
    

# Make sure each image path has one text
list_image_path = []
list_txt = []
for item in input_data:
  if 'filename' in item and 'captions' in item:
    img_path = os.path.join('training', item['filename'].split('/')[-1])
    # cleanup the text string of the caption(s)
    caption = clean_caption(item['captions'])
    # appending path to image then the corresponding caption
    list_image_path.append(img_path)
    list_txt.append(caption)

dataset = image_title_dataset(list_image_path, list_txt)
train_dataloader = DataLoader(dataset, batch_size=100, shuffle=True, num_workers=2) #Define your own dataloader

# Gradient accumulation steps
accumulation_steps = 2

# Adjust learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5 * accumulation_steps)

loss_img = torch.nn.CrossEntropyLoss()
loss_txt = torch.nn.CrossEntropyLoss()

num_epochs = 30
for epoch in range(num_epochs):
    pbar = tqdm(train_dataloader, total = len(train_dataloader))
    for i, batch in enumerate(pbar):
        if batch is None:
            continue
        images,texts = batch
        inputs = processor(texts, images=images, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)

        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text

        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2

        # Normalize loss
        loss = loss / accumulation_steps

        loss.backward()

        # Optimizer step and zero the gradients every accumulation_steps
        if (i+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item():.4f}")
       # model.save_pretrained(f'./model_after_epoch_{epoch}')
        torch.save(model.module.state_dict(), f'./model_after_epoch_{epoch}')
    # If the number of batches is not exactly divisible by accumulation_steps,
    # make sure to still zero the gradients after finishing an epoch
    if len(train_dataloader) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()


Epoch 1/30, Loss: 2.3028:  40%|████      | 6/15 [00:51<01:15,  8.42s/it]

In [ ]:
## and now, we do a little sleight of hand to get the resulting model
## reshaped for how we want to use it
## this is because the dataParallel method can't use save_pretrained.

# change X below:
# Load weights into the model for other tasks
base_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
base_model.load_state_dict(torch.load('model_after_epoch_X.pt'))

base_model.save_pretrained(f'./archae_ai')

In [ ]:
!zip test.zip -r archae_ai/

updating: model_after_epoch_0/ (stored 0%)
  adding: model_after_epoch_0/pytorch_model.bin (deflated 7%)
  adding: model_after_epoch_0/config.json (deflated 46%)
